In [3]:
import pandas as pd,seaborn as sns, numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
data = pd.read_csv('/kaggle/input/ice-cream-dataset/combined/products.csv')

In [5]:
data.head()

,brand,key,name,subhead,description,rating,rating_count,ingredients
0,bj,0_bj,Salted Caramel Core,Sweet Cream Ice Cream with Blonde Brownies & a...,Find your way to the ultimate ice cream experi...,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,1_bj,Netflix & Chilll'd™,Peanut Butter Ice Cream with Sweet & Salty Pre...,There’s something for everyone to watch on Net...,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,2_bj,Chip Happens,A Cold Mess of Chocolate Ice Cream with Fudge ...,Sometimes “chip” happens and everything’s a me...,4.7,130,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3_bj,Cannoli,Mascarpone Ice Cream with Fudge-Covered Pastry...,As a Limited Batch that captured the rapture o...,3.6,70,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4_bj,Gimme S’more!™,Toasted Marshmallow Ice Cream with Chocolate C...,It’s a gimme: there’s always room for s’more. ...,4.5,281,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."


In [5]:
data.drop(['key','subhead','description','name'],axis=1,inplace=True)

In [17]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
rating,241.0,4.223237,0.620383,1.2,4.0,4.4,4.7,5.0
rating_count,241.0,90.016598,111.349449,2.0,28.0,59.0,112.0,983.0


In [6]:
data = data.drop(index=213).reset_index(drop=True)

In [7]:
data.drop('rating_count',axis=1,inplace=True)

In [8]:
#one hot

df = pd.get_dummies(data=data,columns=["brand"],prefix ="is")

In [9]:
df

,rating,ingredients,is_bj,is_breyers,is_hd,is_talenti
0,3.7,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",1,0,0,0
1,4.0,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",1,0,0,0
2,4.7,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,...",1,0,0,0
3,3.6,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",1,0,0,0
4,4.5,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ...",1,0,0,0
...,...,...,...,...,...,...
235,4.0,"MILK, CORN SYRUP, SUGAR, BROWN SUGAR, SOYBEAN ...",0,1,0,0
236,4.7,"MILK, WATER, CARAMEL SWIRL, SUGAR, WATER, CORN...",0,1,0,0
237,2.5,"MILK, CORN SYRUP, SUGAR, WHEAT FLOUR, BUTTER, ...",0,1,0,0
238,3.2,"MILK, CORN SYRUP, ENRICHED WHEAT FLOUR, WHEAT ...",0,1,0,0


# NLP

In [10]:
import nltk
from nltk.corpus import stopwords
from textblob import Word
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [11]:
#remove punktions
df.ingredients = df.ingredients.str.replace("[^\w\s]","")

In [12]:
#sayilarin kaldirilmasi
df.ingredients = df.ingredients.str.replace("\d","")

In [13]:
#Harflerin küçültülmesi
df.ingredients = df.ingredients.apply(lambda x: " ".join(x.lower() for x in x.split()))

In [14]:
#removing stopwords
stopwords = stopwords.words("english")
df.ingredients = df.ingredients.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

In [15]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [16]:
def lemmatize_word(word):
    return nlp(word)[0].lemma_

In [17]:
df['ingredients'] = df['ingredients'].apply(lambda x: ' '.join([lemmatize_word(word) for word in x.split()]))

In [21]:
y = df.rating
X = df.ingredients

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [51]:
from sklearn.linear_model import Lasso

lr = Lasso(alpha=0.01).fit(X_train_tfidf,y_train)

In [52]:
lr.score(X_test_tfidf,y_test)

-0.0042381129942064

In [36]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(lr.predict(X_test_tfidf),y_test))

0.5935591427071629

In [47]:
lr.predict(X_test_tfidf)[60:90]

array([4.27057481, 4.28593336, 4.29848923, 4.27057481, 4.27057481,
       4.27057481, 4.27057481, 4.27057481, 4.27057481, 4.29661316,
       4.27057481, 4.29030677])

In [48]:
y_test[60:90]

95     3.4
126    3.9
120    5.0
67     2.6
101    5.0
68     3.3
108    4.0
84     4.8
117    4.4
194    4.0
177    4.0
111    3.4
Name: rating, dtype: float64